In [11]:
# This code will export summaries of good fire data for each year
# based on assets stored in Google Earth Engine, as well as flattened rasters

In [12]:
import os
from functools import reduce

import ee
import geemap

from find_set_root import find_set_project_root
PROJECT_ROOT = find_set_project_root()
print(f"Project root found at: {PROJECT_ROOT}")
import utils.gee_functions as ugeef

ee.Initialize()

# USER-SET PARAMETERS
EXPORT_FOLDER = 'GEE_Exports'
VERSION = 'welty_wildfire_fri_splitfrg5'
PROJECT_DIR = "projects/ee-tymc5571-goodfire/assets/"


# Load datasets
fires_raw = ee.FeatureCollection("projects/ee-tymc5571-goodfire/assets/welty_wildfire_west_1984_2020")
fire_year_attribute = 'Fire_Year'

gf_cbi = ee.Image("projects/ee-tymc5571-goodfire/assets/welty_wildfire_cbi_bc_1985_2020")


# Summarizing Features
states = ee.FeatureCollection('TIGER/2018/States')
western_states_names = [
    'Washington', 'Oregon', 'California', 'Idaho', 'Nevada', 'Montana',
    'Wyoming', 'Utah', 'Colorado', 'Arizona', 'New Mexico'
]

SUMMARIZE_FEATURES = states.filter(ee.Filter.inList('NAME', western_states_names))
SUMMARIZE_NAME = 'states'

VIZ_RASTER_AOI = states.filter(ee.Filter.eq('NAME', 'California')).geometry().bounds()
VIZ_RASTER_AOI_NAME = 'california'

# Reporting years
earliest_reporting_year = ee.Number(2010).getInfo()  
latest_reporting_year = ee.Number(2020).getInfo()  


# Set up exports
EXPORT_ANNUAL_SUMMARIES = True
EXPORT_EVENT_STATISTICS = False
EXPORT_FLATTENED_RASTER = False

# # This line allows for exporting of specific annual summary years if there are failures on the GEE server side.
# # Comment this line of code out to run all.
#target_years = [2014]

Project root found at: C:\Users\tymc5571\dev\a-number-on-good-fire


In [13]:
# Load GF data

# Get assets in the specified project directory
assets = ugeef.list_assets(PROJECT_DIR)

# Get assets of interest (those containing the version and "cbi_bc_batch_")
gf_assets = [s for s in assets if VERSION in s and "all_gf" in s]
sorted_gf_assets = sorted(gf_assets)
print(sorted_gf_assets)

#Merge
all_gf = ee.ImageCollection.fromImages([ee.Image(a) for a in sorted_gf_assets])

#Convert year attribute to number (stored as string in assets)
all_gf = all_gf.map(lambda img: img.set(
    'year', ee.Number.parse(img.get('year'))
))

years = all_gf.aggregate_array('year').getInfo()
print(years)

# Filter by reporting years
all_gf_reporting_years = all_gf.filter(
    ee.Filter.And(
        ee.Filter.gte('year', earliest_reporting_year),
        ee.Filter.lte('year', latest_reporting_year)
    )
)

all_gf_reporting_years.filter(ee.Filter.eq('year', 2010)).first().bandNames().getInfo()

['projects/ee-tymc5571-goodfire/assets/all_gf_welty_wildfire_fri_splitfrg5_2010', 'projects/ee-tymc5571-goodfire/assets/all_gf_welty_wildfire_fri_splitfrg5_2011', 'projects/ee-tymc5571-goodfire/assets/all_gf_welty_wildfire_fri_splitfrg5_2012', 'projects/ee-tymc5571-goodfire/assets/all_gf_welty_wildfire_fri_splitfrg5_2013', 'projects/ee-tymc5571-goodfire/assets/all_gf_welty_wildfire_fri_splitfrg5_2014', 'projects/ee-tymc5571-goodfire/assets/all_gf_welty_wildfire_fri_splitfrg5_2015', 'projects/ee-tymc5571-goodfire/assets/all_gf_welty_wildfire_fri_splitfrg5_2016', 'projects/ee-tymc5571-goodfire/assets/all_gf_welty_wildfire_fri_splitfrg5_2017', 'projects/ee-tymc5571-goodfire/assets/all_gf_welty_wildfire_fri_splitfrg5_2018', 'projects/ee-tymc5571-goodfire/assets/all_gf_welty_wildfire_fri_splitfrg5_2019', 'projects/ee-tymc5571-goodfire/assets/all_gf_welty_wildfire_fri_splitfrg5_2020']
[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]


['frg1Good',
 'frg2Good',
 'frg3Good',
 'frg4Good',
 'frg5lGood',
 'frg5hGood',
 'tooFrequentLow',
 'tooFrequentHigh',
 'lowerRegimeCbiHigh',
 'replaceRegimeCbiLow',
 'lowerRegimeCbiUnburned',
 'replaceRegimeCbiUnburned',
 'lowerGoodFire',
 'highGoodFire',
 'goodFireAll',
 'cbiAnyBurned',
 'unmatchedRegime',
 'tooFrequent',
 'yearPriorForest',
 'totalArea']

In [14]:
# Generate flattened result

# /////////////////////////////////////////////////////////////////////////////////
# /////////////////////////////////////////////////////////////////////////////////
# ///////////    GENERATE & EXPORT FLATTENED IMAGE FOR REPORTING YEARS   ///////////////
# /////////////////////////////////////////////////////////////////////////////////
# /////////////////////////////////////////////////////////////////////////////////


if EXPORT_FLATTENED_RASTER:
    print("Exporting flattened raster for reporting years...")

    # Mosaic over years and flatten into binary presence layers
    lg_all_reporting_years = all_gf_reporting_years.select('lowerGoodFire').mosaic().gt(0).unmask(0)
    hgf_all_reporting_years = all_gf_reporting_years.select('highGoodFire').mosaic().gt(0).unmask(0)
    unmatched_regime_all_reporting_years = all_gf_reporting_years.select('unmatchedRegime').mosaic().gt(0).unmask(0)
    too_frequent_all_reporting_years = all_gf_reporting_years.select('tooFrequent').mosaic().gt(0).unmask(0)

    # Combine non-good fire types
    not_gf_all_reporting_years = unmatched_regime_all_reporting_years.bitwiseOr(too_frequent_all_reporting_years)

    # Assign values:
    # 3 = not good fire, 2 = high good fire, 1 = low good fire
    not_gf_all_reporting_years = not_gf_all_reporting_years.where(not_gf_all_reporting_years.eq(1), 3)
    hgf_all_reporting_years = hgf_all_reporting_years.where(hgf_all_reporting_years.eq(1), 2)

    # Combine the three into a final classified image
    all_reporting_years_gf_flat = lg_all_reporting_years.max(hgf_all_reporting_years).max(not_gf_all_reporting_years).selfMask()

    # Export flat image for visualization in figure
    gf_flat_task = ee.batch.Export.image.toDrive(
        image=all_reporting_years_gf_flat,
        description=f'gf_data_flatraster_{VERSION}_{VIZ_RASTER_AOI_NAME}_{earliest_reporting_year}_{latest_reporting_year}',
        folder=EXPORT_FOLDER,
        fileNamePrefix=f'gf_data_flatraster_{VERSION}_{VIZ_RASTER_AOI_NAME}_{earliest_reporting_year}_{latest_reporting_year}',
        region=VIZ_RASTER_AOI,
        scale=30,
        crs='EPSG:5070',
        maxPixels=1e13
    )
    gf_flat_task.start()


In [15]:

# GENERATE AND EXPORT ANNUAL SUMMARIES

if EXPORT_ANNUAL_SUMMARIES:

    # Loop over each year
    for year in range(earliest_reporting_year, latest_reporting_year + 1):

        if 'target_years' in globals():
            if year not in target_years:
                continue

        #year_str = ee.Number(year).format()
        
        # Get the image for this year
        img = all_gf_reporting_years.filter(ee.Filter.eq('year', year)).first()

        # Reduce image over each feature in summarizeFeatures
        def reduce_feature(feature):
            reduction = img.reduceRegion(
                reducer=ee.Reducer.sum(),
                geometry=feature.geometry(),
                scale=30,
                tileScale=16,
                maxPixels=1e11
            )
            return feature.set(reduction)\
                        .set('year', year)\
                        .set('units', 'm^2')\
                        .setGeometry(None)

        # Map over feature collection
        mapped_reduction = SUMMARIZE_FEATURES.map(reduce_feature)

        # Set up export task
        summary_task = ee.batch.Export.table.toDrive(
            collection=mapped_reduction,
            description=f'gf_data_{year}_{SUMMARIZE_NAME}_{VERSION}',
            fileNamePrefix=f'gf_data_{year}_{SUMMARIZE_NAME}_{VERSION}',
            folder=EXPORT_FOLDER,
            fileFormat='CSV'
        )
        summary_task.start()
        print(f'Started summary export for year {year}')


# Export fire event-level summaries

if EXPORT_EVENT_STATISTICS:
    
    # Filter fire events by year attribute
    fires_raw_filt = fires_raw.filter(
        ee.Filter.And(
            ee.Filter.gte(fire_year_attribute, earliest_reporting_year),
            ee.Filter.lte(fire_year_attribute, latest_reporting_year)
        )
    )

    # Define the per-fire reduction function
    def gf_reduce_feature(f):
        # Extract year from fire feature
        f_yr = ee.Number(f.get(fire_year_attribute))
        
        # Get matching year image from all_gf
        yr_img = ee.Image(all_gf_reporting_years.filter(ee.Filter.eq('year', f_yr)).first())

        # Reduce over fire geometry
        reduction = yr_img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=f.geometry(),
            scale=30,
            tileScale=16,
            maxPixels=1e11
        )

        # Attach results and remove geometry for CSV
        return f.set(reduction).setGeometry(None)

    # Apply the reducer to each fire
    full_dats_fires = fires_raw_filt.map(gf_reduce_feature)

    # Export as CSV to Google Drive
    event_task = ee.batch.Export.table.toDrive(
        collection=full_dats_fires,
        description=f'gf_data_fire_events_{earliest_reporting_year}_{latest_reporting_year}_{VERSION}',
        fileNamePrefix=f'gf_data_fire_events_{earliest_reporting_year}_{latest_reporting_year}_{VERSION}',
        folder=EXPORT_FOLDER,
        fileFormat='CSV'
    )

    event_task.start()
    print('Started export of fire event data.')

Started summary export for year 2010
Started summary export for year 2011
Started summary export for year 2012
Started summary export for year 2013
Started summary export for year 2014
Started summary export for year 2015
Started summary export for year 2016
Started summary export for year 2017
Started summary export for year 2018
Started summary export for year 2019
Started summary export for year 2020
